In [42]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import torch

# Epsilon Metric

In [43]:
def load_pkl_dict(filename):
    folder = "../experiment_data/"
    with open(folder + filename, "rb") as f:
        loaded_file = pickle.load(f)
        return loaded_file

In [62]:
inspect = load_pkl_dict("oscr_data_base_1_1_-1_ABS_REV_ACTV_N-CLASSES.pkl")[0]
inspect = inspect['1500-1.25']
ccr = inspect[0]
fpr = inspect[1]
print(f"CCR (Y-axis): {ccr}")
print(f"FPR (X-axis) non-log: {fpr}")
print(f"FPR (X-axis) log: {np.log(fpr)}")
print(f"Shape: CCR {ccr.shape} and FPR {fpr.shape}")

threshy = 0.6
ar_diff = np.abs(fpr - threshy)
idx = np.argmin(ar_diff)
print(f"At threshold {threshy} actual value in fpr {fpr[idx]} with corresponding CCR {ccr[idx]}")

def oscr_epsilon_ccr_at_fpr(ccr, fpr, fpr_thresholds):
    threshold_values = []
    for threshold in fpr_thresholds:
        idx = np.where(fpr==threshold)
        value = ccr[idx]
        if value.shape[0] == 0: 
            value = np.append(value, 0.0)
        threshold_values.append((threshold, value))
    return np.sum([t[1] for t in threshold_values]), threshold_values

t_list = [0.1, 0.2, 0.3, 0.4, 0.8]

eps, t_val = oscr_epsilon_ccr_at_fpr(ccr, fpr, t_list)
eps, t_val

CCR (Y-axis): [0.9881 0.9881 0.9881 ... 0.7871 0.7838 0.7756]
FPR (X-axis) non-log: [1.         0.99988636 0.99977273 ... 0.25272727 0.24954545 0.24306818]
FPR (X-axis) log: [ 0.00000000e+00 -1.13642821e-04 -2.27298558e-04 ... -1.37544435e+00
 -1.38811420e+00 -1.41441329e+00]
Shape: CCR (6385,) and FPR (6385,)
At threshold 0.6 actual value in fpr 0.6 with corresponding CCR 0.9753


(2.7737,
 [(0.1, array([0.])),
  (0.2, array([0.])),
  (0.3, array([0.8469])),
  (0.4, array([0.943])),
  (0.8, array([0.9838]))])

In [45]:
# Assume x is your [1000, 50] tensor
x = torch.randn(1000, 50)
print(x[0])
# Reshape the tensor to [1000, 10, 5]
x_reshaped = x.view(1000, 10, 5)
print(x_reshaped[0])

tensor([-0.0756, -0.0340,  0.3358,  0.6007,  1.4918, -0.3603,  0.2670,  0.2639,
        -0.5271, -0.4247, -1.2626, -0.1973,  0.0466,  0.7138,  0.7984, -0.9028,
         0.1227,  0.4770, -0.2349,  0.5676, -0.1189,  0.9912, -2.3174,  0.2938,
         0.3934,  0.0088,  0.4089,  1.4416, -0.1337,  0.2456, -0.0746, -2.0485,
         0.3700,  0.4416, -0.2060,  0.3134, -0.8629,  0.3161,  0.2115, -0.0644,
         0.4713,  1.2420,  0.4874,  0.4133,  0.1439, -2.3594, -0.0884,  0.9521,
         0.5041, -1.7152])
tensor([[-0.0756, -0.0340,  0.3358,  0.6007,  1.4918],
        [-0.3603,  0.2670,  0.2639, -0.5271, -0.4247],
        [-1.2626, -0.1973,  0.0466,  0.7138,  0.7984],
        [-0.9028,  0.1227,  0.4770, -0.2349,  0.5676],
        [-0.1189,  0.9912, -2.3174,  0.2938,  0.3934],
        [ 0.0088,  0.4089,  1.4416, -0.1337,  0.2456],
        [-0.0746, -2.0485,  0.3700,  0.4416, -0.2060],
        [ 0.3134, -0.8629,  0.3161,  0.2115, -0.0644],
        [ 0.4713,  1.2420,  0.4874,  0.4133,  0.1439]

In [46]:
# Use torch.max on the last dimension
output, _ = torch.max(x_reshaped, dim=2)
output[0]

tensor([1.4918, 0.2670, 0.7984, 0.5676, 0.9912, 1.4416, 0.4416, 0.3161, 1.2420,
        0.9521])

In [47]:
def multiply_tensors_with_sign(sorted_activations, weights):
    mask = sorted_activations < 0
    weights[mask] = 2 - weights[mask]
    return sorted_activations * weights

In [48]:
def value_shift(sorted_activations):
    min_values = torch.min(sorted_activations, dim=1).values

    # Reshape the min_values tensor to match the shape of 'a' for broadcasting
    min_values_reshaped = min_values.view(-1, 1)
    min_values_reshaped_abs = torch.abs(min_values_reshaped)

    # Add the minimum values to the corresponding rows of the original tensor
    return torch.add(sorted_activations, min_values_reshaped_abs)

In [49]:
randy = torch.randn(5,5)
randy

tensor([[-1.4965, -1.2885, -0.9408, -0.9227, -0.5991],
        [-1.2427,  0.4799, -0.8002,  1.4641,  0.2359],
        [ 0.9744,  1.3617,  0.0708, -0.2328, -1.2188],
        [-0.0824,  0.1379,  0.7257, -1.2530,  1.0863],
        [ 0.8190,  0.4117,  0.4991,  0.0171,  1.8680]])

In [50]:
rando = torch.rand(5, 5)
rando

tensor([[0.8949, 0.5619, 0.5099, 0.4853, 0.5544],
        [0.5174, 0.4905, 0.7997, 0.7572, 0.3770],
        [0.0502, 0.6436, 0.8028, 0.6691, 0.6133],
        [0.5260, 0.8083, 0.0380, 0.0293, 0.8745],
        [0.9453, 0.6798, 0.3470, 0.6468, 0.5551]])

In [51]:
val = value_shift(randy) * rando

In [52]:
abs_value = multiply_tensors_with_sign(randy, rando)

In [53]:
softmax_abs = torch.nn.functional.softmax(abs_value, dim=1)
abs_value, softmax_abs, torch.max(softmax_abs, dim=1)

(tensor([[-1.6537, -1.8530, -1.4020, -1.3975, -0.8661],
         [-1.8425,  0.2354, -0.9605,  1.1085,  0.0889],
         [ 0.0489,  0.8764,  0.0568, -0.3098, -1.6901],
         [-0.1215,  0.1115,  0.0275, -2.4694,  0.9500],
         [ 0.7742,  0.2799,  0.1732,  0.0111,  1.0369]]),
 tensor([[0.1516, 0.1242, 0.1950, 0.1959, 0.3333],
         [0.0267, 0.2134, 0.0645, 0.5110, 0.1843],
         [0.1934, 0.4425, 0.1950, 0.1351, 0.0340],
         [0.1553, 0.1961, 0.1803, 0.0148, 0.4535],
         [0.2548, 0.1554, 0.1397, 0.1188, 0.3313]]),
 torch.return_types.max(
 values=tensor([0.3333, 0.5110, 0.4425, 0.4535, 0.3313]),
 indices=tensor([4, 3, 1, 4, 4])))

In [54]:
softmax_val = torch.nn.functional.softmax(val, dim=1)
val, softmax_val, torch.max(softmax_val, dim=1)

(tensor([[0.0000, 0.1169, 0.2833, 0.2785, 0.4975],
         [0.0000, 0.8449, 0.3539, 2.0495, 0.5575],
         [0.1100, 1.6609, 1.0354, 0.6597, 0.0000],
         [0.6157, 1.1242, 0.0751, 0.0000, 2.0458],
         [0.7904, 0.2915, 0.1791, 0.0221, 1.0464]]),
 tensor([[0.1558, 0.1751, 0.2069, 0.2059, 0.2563],
         [0.0701, 0.1632, 0.0999, 0.5444, 0.1224],
         [0.0920, 0.4339, 0.2321, 0.1595, 0.0824],
         [0.1256, 0.2088, 0.0731, 0.0678, 0.5247],
         [0.2561, 0.1555, 0.1390, 0.1188, 0.3308]]),
 torch.return_types.max(
 values=tensor([0.2563, 0.5444, 0.4339, 0.5247, 0.3308]),
 indices=tensor([4, 3, 1, 4, 4])))

In [55]:
total = torch.sum(val, dim=1)
total

tensor([1.1761, 3.8057, 3.4660, 3.8608, 2.3295])

In [56]:
matrix = np.arange(25).reshape((5,5))
matrix

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [57]:
maxi = np.max(matrix, axis=1)
maxi

array([ 4,  9, 14, 19, 24])

In [58]:
summe = np.sum(maxi)
summe

70

In [59]:
gt = np.arange(matrix.shape[0])
mask = gt > 0
mask, gt[mask], matrix[mask]

(array([False,  True,  True,  True,  True]),
 array([1, 2, 3, 4]),
 array([[ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24]]))

In [60]:
# Retrieve the values
values = matrix[mask][np.arange(matrix[mask].shape[0]), gt[mask]]

print(values)

[ 6 12 18 24]
